In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

In [2]:
def user_process(data_user, Train=1):
    data = data_user.copy()
    if Train:
        da = data.iloc[:, 4:-1]
        avg_data = da.mean(axis=1)
        avg_data.fillna(avg_data.mean(), inplace=True)
        data['avgfee'] = avg_data
        data['city_name'].fillna('无', inplace=True)
        data['county_name'].fillna('无', inplace=True)
        data = data.drop(['arpu_201908', 'arpu_201909', 'arpu_201910', 'arpu_201911',
                          'arpu_201912', 'arpu_202001', 'arpu_202002', 'arpu_202003'], axis=1)
    else:
        data['arpu_202004'] = data['arpu_202004'].fillna(data['arpu_202004'].mean())
        data['avgfee'] = data['arpu_202004']
        data['city_name'].fillna('无', inplace=True)
        data['county_name'].fillna('无', inplace=True)
        data = data.drop('arpu_202004', axis=1)
    
    return data

In [ ]:
lo_data = data_voc.iloc[:120000, :]
def voc_process(data_voc):
    
    lo_data = data_voc.copy()
    lo_data['start_datetime'] = pd.to_datetime(lo_data['start_datetime'], format='%Y/%m/%d %H:%M:%S')
    lo_data['month'] = lo_data['start_datetime'].dt.month
    
    """
    平均每月总的电话次数
    """
    lo_data['cnt'] = 1
    lo_data.columns
    sum_of_all = pd.pivot_table(lo_data,
                                index=['phone_no_m', 'month'],
                                values='cnt',
                                aggfunc=sum).reset_index(level=['phone_no_m', 'month'])
    avg_sum_of_call = pd.pivot_table(sum_of_all, index=['phone_no_m'], values='cnt',
                                     aggfunc=np.mean).reset_index(level=['phone_no_m'])
    data = avg_sum_of_call.copy()
    
    """
    平均每月电话总时长（分钟）
    """
    lo_data.columns
    sum_of_call_time = (pd.pivot_table(lo_data,
                                       index=['phone_no_m', 'month'],
                                       values='call_dur',
                                       aggfunc=sum) / 60).reset_index(level=['phone_no_m', 'month'])
    sum_of_call_time = pd.pivot_table(sum_of_call_time,
                                      index=['phone_no_m'],
                                      values='call_dur',
                                      aggfunc=np.mean).reset_index(level=['phone_no_m'])
    
    data = pd.merge(data, sum_of_call_time, on='phone_no_m', how='left')
    
    
    """
    统计主叫次数，被叫次数，传呼的次数以及占比
    """
    lo_data['calltype_id_1'] = (lo_data['calltype_id'] == 1).astype(int)
    lo_data['calltype_id_2'] = (lo_data['calltype_id'] == 2).astype(int)
    lo_data['calltype_id_3'] = (lo_data['calltype_id'] == 3).astype(int)
    
    sum_of_call_calltype_id_1 = pd.pivot_table(lo_data,
                                               index=['phone_no_m', 'month'],
                                               values='calltype_id_1',
                                               aggfunc=sum).reset_index(level='phone_no_m', 'month')
    sum_of_call_calltype_id_2 = pd.pivot_table(lo_data,
                                               index=['phone_no_m', 'month'],
                                               values='calltype_id_2',
                                               aggfunc=sum).reset_index(level='phone_no_m', 'month')
    sum_of_call_calltype_id_3 = pd.pivot_table(lo_data,
                                               index=['phone_no_m', 'month'],
                                               values='calltype_id_3',
                                               aggfunc=sum).reset_index(level='phone_no_m', 'month')
    
    sum_of_call_calltype_id_1 = pd.pivot_table(sum_of_call_calltype_id_1,
                                               index=['phone_no_m'],
                                               values='calltype_id_1',
                                               aggfunc=np.mean).reset_index(level=['phone_no_m'])
    sum_of_call_calltype_id_2 = pd.pivot_table(sum_of_call_calltype_id_2,
                                               index=['phone_no_m'],
                                               values='calltype_id_2',
                                               aggfunc=np.mean).reset_index(level=['phone_no_m'])
    sum_of_call_calltype_id_3 = pd.pivot_table(sum_of_call_calltype_id_3,
                                               index=['phone_no_m'],
                                               values='calltype_id_3',
                                               aggfunc=np.mean).reset_index(level=['phone_no_m'])
    
    data = pd.merge(data, sum_of_call_calltype_id_1, on='phone_no_m', how='left')
    data = pd.merge(data, sum_of_call_calltype_id_2, on='phone_no_m', how='left')
    data = pd.merge(data, sum_of_call_calltype_id_3, on='phone_no_m', how='left')
    
    # 占比
    data['sum_of_call_calltype_id'] = data['calltype_id_1'] + data['calltype_id_2'] + data['calltype_id_3']
    data['calltype_id_1_rate'] = data['calltype_id_1'] / data['sum_of_call_calltype_id']
    data['calltype_id_2_rate'] = data['calltype_id_2'] / data['sum_of_call_calltype_id']
    data['calltype_id_3_rate'] = data['calltype_id_3'] / data['sum_of_call_calltype_id']
    
    
    """
    对多少个不同的号码打了电话
    """
    lo_data['sum_of_different_call'] = 1
    sum_of_different_call = pd.pivot_table(lo_data,
                                           index=['phone_no_m', 'opposite_no_m'],
                                           values='sum_of_different_call',
                                           aggfunc=sum).reset_index(level=['phone_no_m', 'opposite_no_m'])
    